# JETTON mass swaps

JETTON had a campaign to incentivize token swaps.
Some users made chains of wallets, with such pattern:
1. Buy JETTON
2. Send all the remaining balance to a new wallet
3. ...
4. PROFIT!

Initial funds came from xRocket. To get the list of all starting wallets the following Tonalytica query 
was used (finds all wallets with only one swaps TON/JETTON and incoming message from xRocket):

```sql
select distinct swap_user from redoubt.dex_swaps dsp where 
swap_dst_token='EQAQXlWJvGbbFfE8F3oS8s87lIgdovS455IsWFaRdmJetTon'
and swap_time  > '2024-08-01'::date
and exists (select from redoubt.messages where 
  source = 'EQABGo8KCza3ea8DNHMnSWZmbRzW-05332eTdfvW-XDQEjQM'
  and destination =swap_user
  and value > 50 * 1e9
)
and (select count(1) from redoubt.dex_swaps dsp2 where dsp2.swap_user  = dsp.swap_user) = 1

```

Total: **770** wallets

In [1]:
start = [
    'EQBEvKOFPHl4RFxH-lg8DwHaTu4lAPvDue63OwT2S0a5kaFh',
    'EQCoo3JqfIibrUzO4Y-Y7i2DVlOar8Qb54E7zB7RNMz-OzfZ',
    'EQCvFGagIMm8vmYwWhgv4__nNiTzxMVYBEWmmQ34xPstUrUi',
    'EQAb6o7v6TqF8nrKPjCu7Gt6YjSU_VJ0VkjUTGE5768i92gT',
    'EQCT2RURRQgiiBkngTaSrg1NhrQDYHj714H1ex9lopY70_mj',
    'EQDGb2praSQTfVdJj5F_Dd180qcO7z3XZC3zVGSe3n6qk-RK',
    'EQDmERtgOhksdV_OtPnReBGdXL2ZSNDTgBvgOhx_VQQn4wLs',
    'EQDvoUjFZH3UGwbBuzGp3sGCju7c8oigqHb3ADZCXp-xlKzN',
    'EQDrDtPuA8sD_qdW4uAkWJOdneI5HOEO_XXte27tpgoYKB69'
]
sybils = set()

In [2]:
import requests 
import time
from tonsdk.utils import Address
from tqdm import tqdm
DEDUST_VAULT = '0:DAE153A74D894BBC32748198CD626E4F5DF4A69AD2FA56CE80FC2644B5708D20'
STONFI_VAULT = '0:1150B518B2626AD51899F98887F8824B70065456455F7FE2813F012699A4061F'

to_check = start
with tqdm() as pbar:
    while True:
        if len(to_check) == 0:
            break

        next_round = set()
        for address in to_check:
            sybils.add(address)
            pbar.update(1)
            res = requests.get(f"https://toncenter.com/api/v3/messages?source={address}").json()
            assert res.get('ok', True)
            has_swap = False
            for m in res['messages']:
                if m['destination'] == DEDUST_VAULT or m['destination'] == STONFI_VAULT:
                    has_swap = True
            if has_swap:
                for m in res['messages']:
                    if m['destination'] != DEDUST_VAULT and m['destination'] != STONFI_VAULT:
                        new_addr = Address(m['destination']).to_string(1, 1, 1)
                        next_round.add(new_addr)


    #         time.sleep(2)
        to_check = next_round

770it [05:49,  2.20it/s]


In [3]:
import pandas as pd
df = pd.DataFrame(sybils, columns=['address']).query("address != ''")
df['project'] = 'JETTON'
df['reason'] = 'mass-accounts'
df.to_csv("jetton_banned.csv", index=False)

In [4]:
from IPython.core.display import display, HTML

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
    display(HTML(df.to_html()))

,address,project,reason
0,EQAc5nMYss4FcNIVDS0sO6tyU_Jaq_OWNYry9TFw7gBjnExF,JETTON,mass-accounts
1,EQCAPBmcV6gL4JRAr0hbCjVWfUbFd9HnfBlrIbb2vQmRhbd4,JETTON,mass-accounts
2,EQCYJYZ4mcLiMzq-IWKxzM1j5bkB12xeDSUjF3xTcVEQst_c,JETTON,mass-accounts
3,EQDW-d6tmIyyVqaWkC0DiHb8BtLGvENxrwXCzoAveEtNVWDZ,JETTON,mass-accounts
4,EQBCI4DUs3De8JwuiiFIUCOE0ytejDlGBhGGFZ8H-TrBB-tD,JETTON,mass-accounts
5,EQAB-ocjHg1B60U-8tvXF3mwpr2C2XFWcQWvGxqXGkosIav9,JETTON,mass-accounts
6,EQCVFwmVWTab_n3IisNQpFjFNkDG1BMgs9wfzF2-jrrwPEJW,JETTON,mass-accounts
7,EQBVnAO36MXUdVWrigOgzSQWkwhOXXaZ69nrPz-alP1fX-vc,JETTON,mass-accounts
8,EQA9vdzoS7026QYmS2ZgQDmfs4c-eAWPMX3EugrtjwF96g4E,JETTON,mass-accounts
9,EQDwmJW2KTP-EEfGpLGe9piK7-_y5XN8hYoLWDYoxeYGScVk,JETTON,mass-accounts
